# Part 1 - Question Answering

For the first part, use the Hugging Face question-answering pipeline and feed it with the five 300-word long sections from the book of your choice that you analyzed in Project 1.

These sections should be selected so they are: introducing the protagonist(s), the antagonist, the crime and crime scene, any significant evidence, and the resolution of the crime/a narrative that presents the case against the perpetrator.

For a prompt, Implement a simple prompt interface that takes in your question, runs it against the model, and returns the answer. You don't need to do anything special about this, just a simple console I/O interface without any complicated error handling. It is up to you how you want to upload the context to the model (pre-loaded into your program, on-demand, etc.).

The questions you should ask are about the identity and characteristics of the protagonist, antagonist/perpetrator, the nature and the setting of the crime or crime scene, the evidence, and the case against the perpetrator.

Document the questions, ask the questions, and document the specificity and accuracy of the results.

Part 1.2 - use two different HF QA models: use the default question-answering pipeline, then use other models of choice and discuss the differences in the result.

https://huggingface.co/docs/transformers/main_classes/pipelines

https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/pipelines#transformers.QuestionAnsweringPipeline


In [1]:
!pip3 install -r ../requirements.txt

%load_ext autoreload
%autoreload 2


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import csv
import os

from src import utils
from src.question_answering import run_qa, run_qa_models


/mnt/Documents/GitHub/UTK-Classes/CS524-Natural-Language-Processing/projects/ChatPI/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
---

## Experiments & Results

For the first part, use the Hugging Face question-answering pipeline and feed it with the five 300-word long sections from the book of your choice that you analyzed in Project 1.

These sections should be selected so they are: **introducing the protagonist(s), the antagonist, the crime and crime scene, any significant evidence, and the resolution of the crime/a narrative that presents the case against the perpetrator.**

The questions you should ask are about the identity and characteristics of the protagonist, antagonist/perpetrator, the nature and the setting of the crime or crime scene, the evidence, and the case against the perpetrator.

Document the questions, ask the questions, and document the specificity and accuracy of the results.


In [3]:
models = [
    # DistilBERT
    "distilbert-base-cased-distilled-squad",
    "distilbert-base-uncased-distilled-squad",
    # RoBERTa
    "deepset/roberta-base-squad2",
    "deepset/roberta-large-squad2",
    # Deberta
    "deepset/deberta-v3-base-squad2",
    "deepset/deberta-v3-large-squad2",
    # Electra
    "deepset/electra-base-squad2",
]


In [4]:
csv_file = "res_qa.csv"
csv_header = [
    "ctx_name",
    "ctx_fname",
    "q_idx",
    "q_text",
    "q_answer_true",
    "qa_model",
    "qa_answer_pred",
]
csv_rows = []

for ctx_name in ["protagonist", "antagonist", "crime", "evidence", "resolution"]:
    for ctx_idx, (ctx_fname, ctx_text) in enumerate(utils.read_context(ctx_name)):
        ctx_fname = os.path.basename(ctx_fname)
        print("#" * 80)
        print("#" * 80)
        print(ctx_text)

        for q_idx, (q_text, q_answer_true) in enumerate(utils.read_qa(ctx_name)):
            print("=" * 80)
            print("=" * 80)
            print(f"Current Question: {q_text}")
            print(f"Expected Answer: {q_answer_true}")

            q_answers_preds, q_answers_scores = run_qa_models(
                q_text,
                ctx_text,
                models,
                q_answer_true,
            )

            for qa_model, qa_answer_pred, qa_answer_score in zip(
                models, q_answers_preds, q_answers_scores
            ):
                row = [
                    ctx_name,
                    ctx_fname,
                    q_idx,
                    q_text,
                    q_answer_true,
                    qa_model,
                    qa_answer_pred,
                ]

                for metric, score in qa_answer_score.items():
                    if metric not in csv_header:
                        csv_header.append(metric)
                    row.append(score)

                csv_rows.append(row)


with open(csv_file, "w") as f:
    writer = csv.writer(f)
    writer.writerow(csv_header)
    writer.writerows(csv_rows)


Found: ['protagonist.0.md', 'protagonist.qa.md']
################################################################################
################################################################################
Sherlock Holmes took his bottle from the corner of the mantel-piece and his hypodermic syringe from its neat morocco case. With his long, white, nervous fingers he adjusted the delicate needle, and rolled back his left shirt-cuff. For some little time his eyes rested thoughtfully upon the sinewy forearm and wrist all dotted and scarred with innumerable puncture-marks. Finally he thrust the sharp point home, pressed down the tiny piston, and sank back into the velvet-lined arm-chair with a long sigh of satisfaction. Three times a day for many months I had witnessed this performance, but custom had not reconciled my mind to it. On the contrary, from day to day I had become more irritable at the sight, and my conscience swelled nightly within me at the thought that I had lacked the

/mnt/Documents/GitHub/UTK-Classes/CS524-Natural-Language-Processing/projects/ChatPI/scratchbooks/src/utils.py:90: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = doc1.similarity(doc2)


{'bertscore_f1': 0.49433884024620056,
 'bertscore_hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.35.0)',
 'bertscore_precision': 0.464594304561615,
 'bertscore_recall': 0.5281525254249573,
 'rouge1': 0.0,
 'rouge2': 0.0,
 'rougeL': 0.0,
 'rougeLsum': 0.0,
 'spacy_sim': 0.0}
--------------------------------------------------------------------------------
model: distilbert-base-uncased-distilled-squad
Q: Who is the main character that the story revolves around?
A: the Beaune (model confidence score: 0.588)
{'bertscore_f1': 0.49187344312667847,
 'bertscore_hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.35.0)',
 'bertscore_precision': 0.48016396164894104,
 'bertscore_recall': 0.5041683316230774,
 'rouge1': 0.0,
 'rouge2': 0.0,
 'rougeL': 0.0,
 'rougeLsum': 0.0,
 'spacy_sim': 0.0}
--------------------------------------------------------------------------------
model: deepset/roberta-base-squad2
Q: Who is the main character th